In [49]:
from pathlib import Path
import pandas as pd
from bs4 import BeautifulSoup
import requests


In [6]:
data_dir = Path("Data")

In [8]:
# Get all files in data_dir directory using pathlib iterdir() method
files = [f for f in data_dir.iterdir() if f.is_file()]

[WindowsPath('Data/1.html'),
 WindowsPath('Data/10.html'),
 WindowsPath('Data/2.html'),
 WindowsPath('Data/3.html'),
 WindowsPath('Data/4.html'),
 WindowsPath('Data/5.html'),
 WindowsPath('Data/6.html'),
 WindowsPath('Data/7.html'),
 WindowsPath('Data/8.html'),
 WindowsPath('Data/9.html')]

In [92]:
def process_linkedin_title(title):
    """
    Function to process the title of a LinkedIn profile
    """
    title = title.replace(" - LinkedIn", "")
    title = title.replace("LinkedIn", "")
    title = title.replace("Professional Profile", "")
    title = title.replace(" at ", ", ")
    title = title.replace(" - ", ", ")
    title = title.replace(" | ", ", ")
    
    title = title.split(", ", 1)
    
    return title[0].strip(), title[1].strip() if len(title) == 2 else ''
    

In [90]:
def process_linkedin_url(url):
    url = url.replace("?trk=public_profile_samename-profile", "")
    url = url.replace("?trk=people-guest_people_search-card", "")
    url = url.replace("?trk=public_profile_browsemap", "")
    url = url.replace("?trk=public_profile_browsemap_profile-result-card_result-card_full-click", "")
    return url

In [95]:
names = []
infos = []
urls = []
    
for f in files:
    # Parse html file using BeautifulSoup and find search results
    # results = BeautifulSoup(f.read_text(), "html.parser").find_all("h3", class_="LC20lb MBeuO DKV0Md")
    
    results = BeautifulSoup(f.read_text(), "html.parser").find_all("div", class_="yuRUbf")
    for r in results:
        # print(r)
        # print('_'*50)
        profile_name = r.find("h3").text
        name, info = process_linkedin_title(profile_name)
        # print('_'*50)
        profile_url = r.find("a", href=True)["href"]
        url = process_linkedin_url(profile_url)
        
        names.append(name)
        infos.append(info)
        urls.append(url)
        
profiles = pd.DataFrame({"Name": names, "Info": infos, "URL": urls})
profiles.to_csv("LinkedIn_profiles.csv", index=False)
    

In [50]:
url = "https://in.linkedin.com/in/falak-naaz-37747420b"

# Read url using BeautifulSoup
raw = BeautifulSoup(requests.get(url).text, "html.parser")
raw

<html><head>
<script type="text/javascript">
window.onload = function() {
  // Parse the tracking code from cookies.
  var trk = "bf";
  var trkInfo = "bf";
  var cookies = document.cookie.split("; ");
  for (var i = 0; i < cookies.length; ++i) {
    if ((cookies[i].indexOf("trkCode=") == 0) && (cookies[i].length > 8)) {
      trk = cookies[i].substring(8);
    }
    else if ((cookies[i].indexOf("trkInfo=") == 0) && (cookies[i].length > 8)) {
      trkInfo = cookies[i].substring(8);
    }
  }

  if (window.location.protocol == "http:") {
    // If "sl" cookie is set, redirect to https.
    for (var i = 0; i < cookies.length; ++i) {
      if ((cookies[i].indexOf("sl=") == 0) && (cookies[i].length > 3)) {
        window.location.href = "https:" + window.location.href.substring(window.location.protocol.length);
        return;
      }
    }
  }

  // Get the new domain. For international domains such as
  // fr.linkedin.com, we convert it to www.linkedin.com
  // treat .cn similar to .com